# How to Develop a Random Forest Ensemble in Python

> Random forest is an ensemble machine learning algorithm.

Es tal vez el algoritmo de *machine learning* más popular y ampliamente usado dado su buen o excelente rendimiento, a lo largo de varios problemas de clasificación y regresión predictiva. También, es sencillo de usar, pues solo tiene algunos hiperparámetros clave y heurísticas sensibles para configurar estos parámetros.

## 1. Random Forest Algorithm

> Random forest is an ensemble of decision tree algorithms.

Es una extensión de **bootstrap aggregation (bagging)** de los árboles de decisión y puede ser utilizado para problemas de clasificación y regresión.

En *bagging*, un número de árboles de decisión son creados, donde cada árbol se crea a partir de una muestra *bootstrap* diferente del *dataset*. Una muestra *bootstrap* es una muestra del *dataset* donde una muestra puede aparecer más de una vez, conocico como **muestreo con reemplazo**.

*Bagging* es un algoritmo de *ensemble* efectivo, pues cada árbol de decisión es entrenado en un *dataset* ligeramente diferente y, por ende, tiene un rendimiento ligeramente diferente. A diferencia de los modelos de árboles de decisión normales, como **classification and regression trees (CART)**, los árboles utilizados en el *ensemble* son *unpruned*, lo que los convierte en ligeramente *overfit* para el *dataset*. Esto es deseable, 

## 2. Random Forest Scikit-Learn API

## 3. Random Forest Hyperparameters

## 4. Common Questions